### SETUP

In [1]:
from datetime import datetime
import pandas as pd 
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [2]:
ml_path =  "data-ml-ms/20241230_Vendas_BR_Mercado_Livre_2024-12-30_21-05hs_1394589919.xlsx"
ms_path =  "data-ml-ms/20241230_Vendas_BR_Mercado_Shops_2024-12-30_21-19hs_1394589919.xlsx"

sales_10 = "data-vendas-internads/VENDAS MES 10 - TODAS VENDAS.csv"
sales_11 = "data-vendas-internads/VENDAS MES 11 - TODAS.csv"
sales_12 = "data-vendas-internads/VENDAS MES 12 - Todas as vendas.csv"

In [3]:
# FUNÇÕES SISTEMA INTERNO
def formatar_valor(valor):
    if isinstance(valor, str):
        # Remove o "R$" e substitui "." por "" (milhar), depois "," por "."
        valor = valor.replace('R$', '').replace('.', '').replace(',', '.')
    return float(valor)

def import_and_format(path):
    df = pd.read_csv(path)
    df['OBS'] = df['OBS'].fillna("-")
    df['Comprador'] = df['Comprador'].fillna("-")
    df['Forma de pagamento'] = df['Forma de pagamento'].fillna("Não Informado")
    df = df.drop(columns=['Tarifas de envio'], errors='ignore')
    df = df.fillna(0)
    df['Comprador'] = df['Comprador'].str.lower()
    
    
    df['Receita por produtos (BRL)'] = df['Receita por produtos (BRL)'].apply(formatar_valor).astype(int)
    return df



def agrupar_por_coluna(dataframe, coluna_agrupamento="CHANNEL"):
    """
    Agrupa o DataFrame por uma coluna específica, calcula a soma de 'Receita por produtos (BRL)',
    e formata os valores no estilo monetário brasileiro.

    Parâmetros:
        dataframe (pd.DataFrame): O DataFrame de entrada.
        coluna_agrupamento (str): O nome da coluna para agrupar. O padrão é "CHANNEL".
    
    Retorna:
        pd.DataFrame: DataFrame agrupado e formatado.
    """
    # Agrupando pela coluna especificada
    bychannel = dataframe.groupby(coluna_agrupamento)[["Receita por produtos (BRL)"]].sum().sort_values(by="Receita por produtos (BRL)")
    
    # Formatando os valores como moeda
    # bychannel['Receita por produtos (BRL)'] = bychannel['Receita por produtos (BRL)'].apply(
    #     lambda x: f"R$ {x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    # )
    
    # Resetando o índice para uma melhor visualização
    bychannel = bychannel.reset_index()

    return bychannel


In [4]:
# Mapeamento de extensões de arquivo para funções de carregamento do pandas
file_formats = {
    "csv": pd.read_csv,
    "xls": pd.read_excel,
    "xlsx": pd.read_excel,
    "xlsm": pd.read_excel,
    "xlsb": pd.read_excel,
}

# Mapeamento de meses em português para seus números correspondentes
meses = {
    'janeiro': '01', 'fevereiro': '02', 'março': '03', 'abril': '04',
    'maio': '05', 'junho': '06', 'julho': '07', 'agosto': '08',
    'setembro': '09', 'outubro': '10', 'novembro': '11', 'dezembro': '12'
}


In [12]:
# Função para carregar dados do arquivo
def load_data(uploaded_file, sheet_name, header=0):
    """
    Carrega o arquivo e retorna um DataFrame do Pandas, com opções específicas para planilhas do Excel.
    """
    try:
        ext = os.path.splitext(uploaded_file.name)[1][1:].lower()  # Extrai a extensão do arquivo
    except AttributeError:
        ext = uploaded_file.split(".")[-1].lower()  # Em caso de erro, extrai pela string do nome

    # Verifica se o formato é suportado
    if ext in file_formats:
        if ext in ['xls', 'xlsx', 'xlsm', 'xlsb']:
            # Para planilhas Excel, carrega a aba específica e ignora as linhas definidas no header
            return pd.read_excel(uploaded_file, sheet_name=sheet_name, header=header)
        else:
            return file_formats[ext](uploaded_file)  # Para outros formatos, usa o mapeamento padrão
    else:
        raise ValueError(f"Formato de arquivo não suportado: {ext}")

# Função para converter datas
def converter_data(data_hora_str):
    """
    Converte uma string de data no formato "d de mês de Y H:M hs." para "d/m/Y".
    """
    for mes, numero in meses.items():
        data_hora_str = data_hora_str.replace(mes, numero)
    try:
        data_hora = datetime.strptime(data_hora_str, "%d de %m de %Y %H:%M hs.")
        return data_hora.strftime("%d/%m/%Y")
    except ValueError:
        return None  # Caso a data seja inválida ou nula

# Função para limpar e processar dados de vendas
def sales_data_cleaning(data, x):
    """
    Limpa e processa os dados de vendas.
    """
    # Processar o DataFrame 'vendas'
    vendas = data.iloc[:, :11]  # Seleciona as primeiras 11 colunas
    vendas['Unidades'] = vendas['Unidades'].fillna(2).astype(int)
    vendas = vendas.drop(
        columns=['Receita por envio (BRL)', 'Status', 'Descrição do status', 'Pacote de diversos produtos'],
        errors='ignore'
    )
    vendas['Receita por produtos (BRL)'] = vendas['Receita por produtos (BRL)'].fillna(
        data['Preço unitário de venda do anúncio (BRL)']
    )
    vendas['Cancelamentos e reembolsos (BRL)'] = vendas['Cancelamentos e reembolsos (BRL)'].fillna(0)
    vendas['Tarifa de venda e impostos'] = vendas['Tarifa de venda e impostos'].fillna(0)
    vendas['Tarifas de envio'] = vendas['Tarifas de envio'].fillna(0)

    # Processar o DataFrame 'compradores'
    compradores = data.iloc[:, [0]].join(data.iloc[:, 26:33])  # Seleciona a primeira coluna e as colunas 26 a 32

    # Converter a data na coluna 'Data da venda'
    vendas['Data da venda'] = data['Data da venda'].apply(converter_data).fillna(0)

    # Mesclar DataFrames 'vendas' e 'compradores'
    df_merged = pd.merge(vendas, compradores, on='N.º de venda', how='inner')

    # Adicionar canal (CHANNEL)
    df_merged['CHANNEL'] = x

    # Adicionar colunas ITEM_ID e TITLE, se existirem no DataFrame original
    if data.shape[1] > 15:  # Verifica se a coluna 15 existe
        df_merged.insert(15, 'ITEM_ID', data.iloc[:, 15])  # Adiciona a coluna do DataFrame original na posição 15
    if data.shape[1] > 16:  # Verifica se a coluna 16 existe
        df_merged.insert(16, 'TITLE', data.iloc[:, 16])  # Adiciona a coluna do DataFrame original na posição 16

    return df_merged

# Função principal para formatar os dados de vendas
def format_sales(data, channel):
    """
    Formata os dados de vendas a partir do arquivo carregado.
    """
    data = load_data(data, "Vendas BR", header=5)  # Passa o arquivo carregado como argumento
    data = sales_data_cleaning(data, channel)
    data = data.rename(columns={"Endereço.1": "Endereço", "Status.1": "UF"})
    return data

### VENDAS INTERNAS

In [15]:
df_10 = import_and_format(sales_10)
df_11 = import_and_format(sales_11)
df_12 = import_and_format(sales_12)


df_list = [df_10, df_11, df_12]

for i in df_list:
    
    print(i.shape)
    
    
df_combined = pd.concat([df_10, df_11, df_12], ignore_index=True)

# Exibindo o DataFrame resultante



(22, 10)
(35, 10)
(89, 14)


In [17]:
df_combined = df_combined.drop(columns=['Orçamento de Envio - 2', 'Rastreio'], errors='ignore')
df_combined['Orçamento de Envio'] = df_combined['Orçamento de Envio'].fillna(0)
df_combined['THIAGO'] = df_combined['THIAGO'].fillna(False)
df_combined['Orçamento de Envio'] = df_combined['Orçamento de Envio'].apply(formatar_valor).astype(int)

C:\Users\nonak\AppData\Local\Temp\ipykernel_19116\1044554880.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_combined['THIAGO'] = df_combined['THIAGO'].fillna(False)


### VENDAS PLATAFORMA

In [81]:
df_ml = format_sales(ml_path,'MERCADOLIVRE')
df_ms = format_sales(ms_path,'MERCADOSHOPS')
print(df_ml.shape)
print(df_ms.shape)

(126, 17)
(124, 17)


In [83]:
df_ms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   N.º de venda                      124 non-null    int64  
 1   Data da venda                     124 non-null    object 
 2   Unidades                          124 non-null    int32  
 3   Receita por produtos (BRL)        124 non-null    float64
 4   Tarifa de venda e impostos        124 non-null    float64
 5   Tarifas de envio                  124 non-null    float64
 6   Cancelamentos e reembolsos (BRL)  124 non-null    float64
 7   Comprador                         124 non-null    object 
 8   CPF                               124 non-null    object 
 9   Endereço                          124 non-null    object 
 10  Cidade                            124 non-null    object 
 11  UF                                124 non-null    object 
 12  CEP     

In [85]:
df_combined = pd.concat([df_ml, df_ms], ignore_index=True)
# Aplicando a formatação
to_format = ['Receita por produtos (BRL)', 'Tarifa de venda e impostos',
       'Tarifas de envio', 'Cancelamentos e reembolsos (BRL)']
for col in to_format:
    df_combined[col] = df_combined[col].apply(formatar_valor).astype(float).abs()


# df_combined['Forma de pagamento'] = df_combined['Forma de pagamento'].apply(lambda x: ' '.join(x.split()))
# df_combined_

### EXPORT

In [88]:
df_combined.columns

Index(['N.º de venda', 'Data da venda', 'Unidades',
       'Receita por produtos (BRL)', 'Tarifa de venda e impostos',
       'Tarifas de envio', 'Cancelamentos e reembolsos (BRL)', 'Comprador',
       'CPF', 'Endereço', 'Cidade', 'UF', 'CEP', 'País', 'CHANNEL', 'ITEM_ID',
       'TITLE'],
      dtype='object')

In [58]:
df_combined.to_csv("exported/VENDAS-INTERNAS.csv", index=False)

In [ ]:
df_combined_2.to_csv("exported/VENDAS-PLATAFORMA.csv", index=False)